## **Biostat 257 Homework 6**

**Due June 10 @ 11:59PM**

In [1]:
# load necessary packages; make sure install them first
using BenchmarkTools, Distributions, LinearAlgebra, Random, Revise

### **Q1. (10 pts) Refresher on normal-normal model**

$$f(\gamma \vert y) \propto f(y \vert \gamma) \times f(\gamma) \\
= \exp \big \{ -\frac{1}{2} \big(y - ( X\beta + Z\gamma)\big)^{T}\sigma^{-2}I \big(y - (X\beta + Z\gamma)\big)\big\} 
\times \exp \{-\frac{1}{2} \gamma^{T} \Sigma^{-1} \gamma \big\} \\
= \exp \big \{ -\frac{1}{2} \big [ (y-(X\beta + Z\gamma))^{T} \sigma^{-2}I(y-(X\beta + Z\gamma)) + \gamma^{T}\Sigma^{-1}\big ]\big\} \\
= \exp \big \{-\frac{1}{2} \big [ (y^{T}\sigma^{-2}I - \beta^{T}X^{T}\sigma^{-2}I - \gamma^{T}Z^{T}\sigma^{-2}I) \times (y-X\beta-Z\gamma) + \gamma^{T}\Sigma^{-1}\gamma \big]\big \}\\
\propto \exp \big \{-\frac{1}{2} \big [-y^{T}\sigma^{-2}Z\gamma + \beta^{T}X^{T}Z\gamma - \gamma Z^{T}\sigma^{-2}y + \gamma^{T}Z^{T}\sigma^{-2}X\beta - \gamma^{T}Z^{T}\sigma^{-2}Z\gamma + \gamma^{T}\Sigma^{-1}\gamma \big]\big\} \\
\exp \big \{-\frac{1}{2} \big [ -\gamma^{T}Z^{T}\sigma^{-2}y + \gamma^{T}Z^{T}\sigma^{-2}X\beta - \gamma^{T}Z^{T}\sigma^{-2}y + \gamma^{T}Z^{T}\sigma^{-2}X\beta + \gamma^{T}Z^{T}\sigma^{-2}Z\gamma + \gamma^{T}\Sigma^{-1}\gamma \big]\big\} \\
= \exp \big \{-\frac{1}{2} \big (-\sigma^{-2}\gamma^{T}Z^{T}(y-X\beta) - \sigma^{-2}\gamma^{T}Z^{T}(y-X\beta) + \gamma^{T}Z^{T}\sigma^{-2}Z\gamma + \gamma^{T}\Sigma^{-1}\gamma \big) \big\} \\
= \exp \big \{ -\frac{1}{2} \big (-2\sigma^{-2}\gamma^{T}Z^{T}(y-X\beta) + \gamma^{T}(\sigma^{-2}Z^{T}Z + \Sigma^{-1})\gamma \big)\big\}
$$ 

By multivariate completion of square, we know that:

$$ \gamma^{T}(\sigma^{-2}Z^{T}Z + \Sigma^{-1})\gamma -2\sigma^{-2}\gamma^{T}Z^{T}(y-X\beta) \propto \\
(\gamma - (\sigma^{-2}Z^{T}Z + \Sigma^{-1})^{-1} \sigma^{-2}Z^{T}(y-X\beta))^{T}(\sigma^{-2}Z^{T}Z + \Sigma^{-1})(\gamma - (\sigma^{-2}Z^{T}Z + \Sigma^{-1})\sigma^{-2}Z^{T}(y-X\beta))$$

Which means:

$$\exp \big \{ -\frac{1}{2} \big (-2\sigma^{-2}\gamma^{T}Z^{T}(y-X\beta) + \gamma^{T}(\sigma^{-2}Z^{T}Z + \Sigma^{-1})\gamma \big)\big\} \propto \\
\exp \big \{ -\frac{1}{2} \big [ (\gamma - (\sigma^{-2}Z^{T}Z + \Sigma^{-1})^{-1}\sigma^{-2}Z^{T}(y-X\beta))^{T}(\sigma^{-2}Z^{T}Z + \Sigma^{-1})(\gamma - (\sigma^{-2}Z^{T}Z + \Sigma^{-1})^{-1}\sigma^{-2}Z^{T}(y-X\beta))\big]\big\}$$

Which is the kernel of a Multivariate Normal Density with:

- **Mean** : $\sigma^{-2}(\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})^{-1}Z_i^{T}(y_i-X_i\beta)$
- **Covariance**: $(\sigma^{-2}Z_i^{T}Z + \Sigma^{-1})^{-1}$

### **Q2. (20 pts) Derive EM algorithm**

**1. Write Down the Complete Log-Likelihood**

$$ \ell(\beta, \Sigma, \sigma^2) = \sum_{i = 1}^{m}-\frac{1}{2}\log \det(2\pi\sigma^2I) - \frac{1}{2}(y_i - (X_i\beta + Z\gamma_i))^{T}(\sigma^2I)(y_i - (X_i\beta + Z\gamma_i)) - \frac{1}{2} \log \det(2\pi\Sigma) -\frac{1}{2} \gamma_i\Sigma^{-1}\gamma_i\\
= \sum_{i = 1}^{m} -\frac{(n_i + q)}{2}\log(2\pi) - \frac{n_i}{2}\log(\sigma^2)-\frac{1}{2}\log\det(\Sigma) - \frac{1}{2\sigma^2}[(y_i - X_i\beta)^{T}(y_i - X_i\beta) - 2(y_i - X_i\beta)^{T}Z_i\gamma_i + \gamma_i^{T}Z_i^{T}Z_i\gamma_i] - \frac{1}{2}\gamma_i^{T}\Sigma^{-1}\gamma_i \\
= \sum_{i = 1}^{m} -\frac{(n_i + q)}{2}\log(2\pi) - \frac{n_i}{2}\log(\sigma^2)-\frac{1}{2}\log\det(\Sigma) - \frac{1}{2\sigma^2}[(y_i - X_i\beta)^{T}(y_i - X_i\beta) - 2(y_i - X_i\beta)^{T}Z_i\gamma_i) -\frac{1}{2}\gamma_i^{T}\sigma^{-2}Z_i^{T}Z_i\gamma_i - \frac{1}{2}\gamma_i^{T}\Sigma^{-1}\gamma_i \\
= \sum_{i = 1}^{m} -\frac{(n_i + q)}{2}\log(2\pi) - \frac{n_i}{2}\log(\sigma^2)-\frac{1}{2}\log\det(\Sigma) - \frac{1}{2\sigma^2}[(y_i - X_i\beta)^{T}(y_i - X_i\beta) - 2(y_i - X_i\beta)^{T}Z_i\gamma_i) -\frac{1}{2}\gamma_i^{T}(\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})\gamma_i
$$

**2. Derive the $Q$ function (E-step).**

- Recall from the first problem: 
$\mathbb{E}(\gamma_i \mid y_i) = \sigma^{-2}(\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})^{-1}Z_i^{T}(y_i-X_i\beta) = \mu_{\gamma \mid y} \\
\mathrm{Var}(\gamma_i \mid y_i) =  (\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})^{-1} = \Sigma_{\gamma \mid y}$
- Recall the following property regarding quadratic forms from 250A: \
If $x \sim N_q(\mu, \Sigma)$ then $\mathbb{E}(x^{T}Ax) = \mathrm{tr}(A\Sigma) + \mu^{T}A\mu$ 
- Putting these two together, we see that: \
$\mathbb{E}(\gamma_i \mid y_i, \beta^{(t)}, \Sigma^{(t)}, \sigma^{2(t)}) = \sigma^{-2(t)}(\sigma^{-2(t)}Z_i^{T}Z_i + \Sigma^{-1(t)})^{-1}Z_i^{T}(y_i-X_i\beta^{(t)}) = \mu^{(t)}_{\gamma \mid y ...}$ \
$\mathrm{Var}(\gamma_i \mid y_i, \beta^{(t)}, \Sigma^{(t)}, \sigma^{2(t)}) =  (\sigma^{-2}Z_i^{T}Z_i + (\Sigma^{(t)})^{-1})^{-1} = \Sigma^{(t)}_{\gamma \mid y ...}$ \
$\mathbb{E}(\gamma_i^{T}(\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})\gamma_i) = \mathrm{tr}[(\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})\Sigma^{(t)}_{\gamma \mid y}] + \mu^{T(t)}_{\gamma \mid y}(\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})\mu^{(t)}_{\gamma \mid y ...}$ 

- Using these results we see that the conditional expectation evaluates to:
$$\sum_{i = 1}^{m} -\frac{(n_i + q)}{2}\log(2\pi) - \frac{n_i}{2}\log(\sigma^2)-\frac{1}{2}\log \det(\Sigma) - \frac{1}{2\sigma^2}[(y_i - X_i\beta)^{T}(y_i - X_i\beta) - 2(y_i - X_i\beta)^{T}Z_i\mu_{\gamma \mid y ...}] - \frac{1}{2}\mathrm{tr}[(\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})\Sigma^{(t)}_{\gamma \mid y ...}] - \frac{1}{2} \mu^{T(t)}_{\gamma \mid y ...}(\sigma^{2}Z_i^{T}Z_i + \Sigma^{-1})\mu^{(t)}_{\gamma \mid y ...} \\
= \sum_{i = 1}^{m} -\frac{(n_i + q)}{2}\log(2\pi) - \frac{n_i}{2}\log(\sigma^2)-\frac{1}{2}\log \det(\Sigma) - \frac{1}{2\sigma^2}[(y_i - X_i\beta)^{T}(y_i - X_i\beta) - 2(y_i - X_i\beta)^{T}Z_i\mu_{\gamma \mid y ...}] - \frac{1}{2}\mathrm{tr}[(\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})\Sigma^{(t)}_{\gamma \mid y ...}] - \frac{1}{2}\mathrm{tr}[\mu^{T(t)}_{\gamma \mid y ...}(\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})\mu^{(t)}_{\gamma \mid y ...}] \\
= \sum_{i = 1}^{m} -\frac{(n_i + q)}{2}\log(2\pi) - \frac{n_i}{2}\log(\sigma^2)-\frac{1}{2}\log \det(\Sigma) - \frac{1}{2\sigma^2}[(y_i - X_i\beta)^{T}(y_i - X_i\beta) - 2(y_i - X_i\beta)^{T}Z_i\mu_{\gamma \mid y ...}] - \frac{1}{2}\mathrm{tr}[(\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})\Sigma^{(t)}_{\gamma \mid y ...}] - \frac{1}{2}\mathrm{tr}[(\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})\mu^{(t)}_{\gamma \mid y ...} \mu^{T(t)}_{\gamma \mid y ...}] \\
= \sum_{i = 1}^{m} -\frac{(n_i + q)}{2}\log(2\pi) - \frac{n_i}{2}\log(\sigma^2)-\frac{1}{2}\log \det(\Sigma) - \frac{1}{2\sigma^2}[(y_i - X_i\beta)^{T}(y_i - X_i\beta) - 2(y_i - X_i\beta)^{T}Z_i\mu_{\gamma \mid y ...}] - \frac{1}{2}\mathrm{tr}[(\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})(\Sigma^{(t)}_{\gamma \mid y ...} + \mu^{(t)}_{\gamma \mid y ...} \mu^{T(t)}_{\gamma \mid y ...}]
$$

- Which leaves us with:
$$ Q = \sum_{i = 1}^{m} -\frac{(n_i + q)}{2}\log(2\pi) - \frac{n_i}{2}\log(\sigma^2)-\frac{1}{2}\log \det(\Sigma) - \frac{1}{2\sigma^2}[(y_i - X_i\beta)^{T}(y_i - X_i\beta) - 2(y_i - X_i\beta)^{T}Z_i\mu_{\gamma \mid y ...}] - \frac{1}{2}\mathrm{tr}[(\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})(\Sigma^{(t)}_{\gamma \mid y ...} + \mu^{(t)}_{\gamma \mid y ...} \mu^{T(t)}_{\gamma \mid y ...}]$$


**2. Derive the EM (or ECM) update of $\mathbf{\beta, \Sigma, \sigma^2}$ .**

- **Gradient with respect to $\beta$:** 
$\nabla_{\beta} = \frac{d}{d\beta}[\sum_{i=1}^{m} -\frac{1}{2\sigma^2}(y_i - X_i\beta)^{T}(y_i - X_i\beta) - 2(y_i - X_i\beta)^{T}Z_i\mu_{\gamma \mid y ...} \\
=\frac{d}{d\beta}[\sum_{i=1}^{m}-\frac{1}{2\sigma^2}(y^{T}_iy_i - 2\beta^{T}X^{T}_iy_i + \beta^{T}X^{T}_iX_i\beta) - 2y_iZ_i\mu_{\gamma \mid y ...} + 2\beta^{T}X^{T}_iZ_i\mu_{\gamma \mid y ...}]\\
= \sum_{i=1}^{m} X^{T}_iy_i - X^{T}_iX_i\beta - X^{T}_iZ_i\mu_{\gamma \mid y ...} \\
= \sum_{i=1}^{m} X_i^{T}(y_i - X_i\beta) + X_iZ_i\mu_{\gamma \mid y ...}
$

- **Gradient with respect to $\sigma^2$:** \
$\nabla_{\sigma^2} = -\frac{n_i}{2} \log(\sigma^2) -\frac{1}{2\sigma^2}(y_i - X_i\beta)^{T}(y_i - X_i\beta) + \frac{1}{\sigma^2}(y_i - X_i\beta)^{T}Z_i\mu_{\gamma \mid y ...} - \frac{1}{2}\mathrm{tr}[(\sigma^{-2}Z_i^{T}Z_i + \Sigma^{-1})(\Sigma^{(t)}_{\gamma \mid y ...} + \mu^{(t)}_{\gamma \mid y ...}\mu^{(t)T}_{\gamma \mid y ...})] \\
=-\frac{n_i}{\sigma^2} + \frac{1}{2\sigma^4}(y_i - X_i\beta)^{T}(y_i - X_i\beta) - \frac{1}{\sigma^4}(y_i - X_i\beta)^{T}Z_i\mu_{\gamma \mid y ...} - \frac{1}{2\sigma^4}\mathrm{tr}[Z_i^{T}Z_i(\Sigma^{(t)}_{\gamma \mid y ...} + \mu^{(t)}_{\gamma \mid y ...} \mu^{(t)T}_{\gamma \mid y ...})]
$


- **Next: To get (t+1) iterated value of $\mathbf{\beta, \sigma^2}$ and $\mathbf{\Sigma}$ we set all of the gradients equal to zero and solve for the variable of interest. This gives us:**
 1. $\beta^{(t + 1)} = (\sum_{i=1}^{m} X_i^{T}X_i)^{-1}(\sum_{i=1}^{m} X_i^{T}y_i - X_i^{T}Z_i\mu_{\gamma \mid y ...})$
 2. 

### Q3. (20 pts) Objective of a single datum

We modify the code from HW5 to evaluate the objective, the conditional mean of  𝜸 , and the conditional variance of  𝜸 . Start-up code is provided below. You do not have to use this code.

In [2]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # posterior mean and variance of random effects γ
    μγ         :: Vector{T} # posterior mean of random effects
    νγ         :: Matrix{T} # posterior variance of random effects
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    rtr        :: Vector{T}
    xty        :: Vector{T}
    zty        :: Vector{T}
    ztr        :: Vector{T}
    ltztr      :: Vector{T}
    xtr        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    ltztzl     :: Matrix{T}
    storage_qq :: Matrix{T}
    storage_qq2:: Matrix{T}
    storage_qq3:: Matrix{T}
    storage_qq4:: Matrix{T}
    storage_qq5:: Matrix{T}
    storage_qq6:: Matrix{T}
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
    y::Vector{T}, 
    X::Matrix{T}, 
    Z::Matrix{T}) where T <: AbstractFloat
    n, p, q = size(X, 1), size(X, 2), size(Z, 2)
    μγ         = Vector{T}(undef, q)
    νγ         = Matrix{T}(undef, q, q)
    yty        = abs2(norm(y))
    rtr        = Vector{T}(undef, 1)
    xty        = transpose(X) * y
    zty        = transpose(Z) * y
    ztr        = similar(zty)
    ltztr      = similar(zty)
    xtr        = Vector{T}(undef, p)
    storage_p  = similar(xtr)
    storage_q  = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    ltztzl     = similar(ztz)
    storage_qq = similar(ztz)
    storage_qq2= similar(ztz)
    storage_qq3= similar(ztz)
    storage_qq4= similar(ztz)
    storage_qq5= similar(ztz)
    storage_qq6=similar(ztz)
    LmmObs(y, X, Z, μγ, νγ, 
        yty, rtr, xty, zty, ztr, ltztr, xtr,
        storage_p, storage_q, 
        xtx, ztx, ztz, ltztzl, storage_qq, storage_qq2, storage_qq3, storage_qq4, storage_qq5, storage_qq6)
end

"""
    logl!(obs::LmmObs, β, Σ, L, σ², updater = false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `Σ`, 
and `σ²`. The lower triangular Cholesky factor `L` of `Σ` must be supplied too.
The fields `obs.μγ` and `obs.νγ` are overwritten by the posterior mean and 
posterior variance of random effects. If `updater==true`, fields `obs.ztr`, 
`obs.xtr`, and `obs.rtr` are updated according to input parameter values. 
Otherwise, it assumes these three fields are pre-computed. 
"""
function logl!(
        obs     :: LmmObs{T}, 
        β       :: Vector{T}, 
        Σ       :: Matrix{T},
        L       :: Matrix{T},
        σ²      :: T,
        updater :: Bool = false
        ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    σ²inv   = inv(σ²)
    ####################
    # Evaluate objective
    ####################
    # form the q-by-q matrix: Lt Zt Z L
    copy!(obs.ltztzl, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.ltztzl) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.ltztzl) # O(q^3)        
    # form the q-by-q matrix: M = σ² I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ltztzl)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # Zt * res
    updater && BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.ztr, obs.zty)) # O(pq)
    # Lt * (Zt * res)
    BLAS.trmv!('L', 'T', 'N', L, copy!(obs.ltztr, obs.ztr))    # O(q^2)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, copy!(obs.storage_q, obs.ltztr)) # O(q^3)
    # Xt * res = Xt * y - Xt * X * β
    updater && BLAS.gemv!('N', T(-1), obs.xtx, β, T(1), copy!(obs.xtr, obs.xty))
    # l2 norm of residual vector
    updater && (obs.rtr[1] = obs.yty - dot(obs.xty, β) - dot(obs.xtr, β))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q # log det term
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (obs.rtr[1] - qf) * σ²inv 
    logl /= -2
    ######################################
    # TODO: Evaluate posterior mean and variance
    
    ### posterior variance (νγ) ###
    
    copy!(obs.storage_qq2, obs.ztz)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.storage_qq2) 
    # ztzL
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.storage_qq2)
    # L'ztzL
    LAPACK.potrs!('U', obs.storage_qq, obs.storage_qq2)
    # (V'V)^{-1} L'ztzL
    
    mul!(obs.storage_qq3, obs.ztz, L) 
    copy!(obs.storage_qq4, obs.storage_qq3)
    BLAS.gemm!('N', 'N', T(1/σ²), obs.storage_qq3, obs.storage_qq2, 
            T(-1/σ²), obs.storage_qq4)
    # 1/σ²*ztzL*(V'V)^{-1} L'ztzL - 1/σ²*ztzL 
        
    # // note: (V'V)^{-1} L'ztzL computed previously, 
    # stored in obs.storage_qq2
    
    mul!(obs.storage_qq5, L, transpose(L))
    copy!(obs.storage_qq6, L)
    BLAS.gemm!('N', 'N',  T(1), obs.storage_qq5, obs.storage_qq4, T(1), obs.storage_qq6)
    mul!(obs.νγ, obs.storage_qq6, transpose(L))
    
    
    ### posterior mean (μγ) ###
    
    BLAS.gemm!('N','N', T(1), obs.νγ, obs.ztr, T(0), obs.μγ)
    obs.μγ ./= σ²
    
    ###################
    # Return
    ###################        
    return logl
end

logl!

It is a good idea to test correctness and efficiency of the single datum objective/posterior mean/var evaluator here. It's the same test datum in HW3 and HW5.

In [3]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X = [ones(n) randn(n, p - 1)]
Z = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

In [5]:
#obs.storage_qq4 # S # -Z'*inv(σ²*I + Z*L*L'*Z')*Z*L

3×3 Matrix{Float64}:
 -0.999236     0.100354    0.0916096
 -7.04315e-5  -1.00426     0.0916357
 -7.14965e-5  -4.0826e-5  -1.00844

In [11]:
#S = -Z'*inv(σ²*I + Z*L*L'*Z')*Z*L # obs.storage_qq4

3×3 Matrix{Float64}:
 -0.999236     0.100354    0.0916096
 -7.04315e-5  -1.00426     0.0916357
 -7.14965e-5  -4.0826e-5  -1.00844

In [14]:
#(L + L*L'S)L'

3×3 Matrix{Float64}:
  0.000749436  -1.21834e-6   -2.17678e-6
 -1.21834e-6    0.000754233   2.15535e-5
 -2.17678e-6    2.15535e-5    0.000746527

In [12]:
#B = L*L'-L*L'*Z'*inv(σ²*I + Z*L*L'*Z')*Z*L*L' # L*L' - L*L' S L'   (L - L*L'S)L'

3×3 Matrix{Float64}:
  0.000749436  -1.21834e-6   -2.17678e-6
 -1.21834e-6    0.000754233   2.15535e-5
 -2.17678e-6    2.15535e-5    0.000746527

#### **3.1  Correctness**

In [4]:
@show logl = logl!(obs, β, Σ, L, σ², true)
@show obs.μγ
@show obs.νγ;

logl = logl!(obs, β, Σ, L, σ², true) = -3256.1793358058258
obs.μγ = [0.10608689298645836, -0.251041906089828, -1.4653979410591285]
obs.νγ = [0.0007494356395767454 -1.2183420378162024e-6 -2.1767836705487566e-6; -1.21834204662441e-6 0.0007542331466978082 2.1553464636468693e-5; -2.1767836415192976e-6 2.15534646046981e-5 0.0007465271345336]


You will lose all 20 points if following statement throws AssertionError.

In [5]:
@assert abs(logl - (-3256.1793358058258)) < 1e-4
@assert norm(obs.μγ - [0.10608689301333621, 
        -0.25104190602577225, -1.4653979409855415]) < 1e-4
@assert norm(obs.νγ - [
        0.0007494356395909563 -1.2183420093769967e-6 -2.176783643112221e-6; 
        -1.2183420282298223e-6 0.0007542331467601107 2.1553464632686345e-5; 
        -2.1767836636008638e-6 2.1553464641863096e-5 0.0007465271342535443
        ]) < 1e-4

#### **3.2  Efficiency**

Benchmark for efficiency.

In [32]:
bm_obj = @benchmark logl!($obs, $β, $Σ, $L, $σ², true)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.034 μs … 587.205 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.158 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.884 μs ±   6.659 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▄▅▆▄▃▂▁▂   ▁▁▁▂▂▃▃▂▂▁▁                                    ▂
  ██████████▇█▇█████████████▇▇▇▇▆▆▆▆▆▅▅▆▅▅▆▅▅▁▅▄▅▅▅▅▄▄▃▅▄▅▅▄▅ █
  2.03 μs      Histogram: log(frequency) by time      8.89 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median run time is 1.8μs. You will get full credit if the median run time is within 10μs. The points you will get are:

In [34]:
clamp(10 / (median(bm_obj).time / 1e3) * 10, 0, 10)

10.0